In [16]:
import pandas as pd
df = pd.read_csv('./datasets/Predict Health Horses.csv')
df.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,...,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1235 non-null   int64  
 1   surgery                1235 non-null   object 
 2   age                    1235 non-null   object 
 3   hospital_number        1235 non-null   int64  
 4   rectal_temp            1235 non-null   float64
 5   pulse                  1235 non-null   float64
 6   respiratory_rate       1235 non-null   float64
 7   temp_of_extremities    1235 non-null   object 
 8   peripheral_pulse       1235 non-null   object 
 9   mucous_membrane        1235 non-null   object 
 10  capillary_refill_time  1235 non-null   object 
 11  pain                   1235 non-null   object 
 12  peristalsis            1235 non-null   object 
 13  abdominal_distention   1235 non-null   object 
 14  nasogastric_tube       1235 non-null   object 
 15  naso

In [18]:
df.isna().sum()

id                       0
surgery                  0
age                      0
hospital_number          0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
nasogastric_reflux_ph    0
rectal_exam_feces        0
abdomen                  0
packed_cell_volume       0
total_protein            0
abdomo_appearance        0
abdomo_protein           0
surgical_lesion          0
lesion_1                 0
lesion_2                 0
lesion_3                 0
cp_data                  0
outcome                  0
dtype: int64

In [19]:
# Define a function to fill null values usaing mode.
def remove_null_using_mode(df, column_name):
    # Get mode value of column 
    mode_value = df[column_name].mode().iloc[0]
    
    # Fill null values using mode 
    df[column_name].fillna(mode_value, inplace=True)
    return True

In [20]:
null_columns = ['temp_of_extremities', 
                'peripheral_pulse',
                'mucous_membrane',
                'capillary_refill_time',
                'pain',
                'peristalsis',
                'abdominal_distention',
                'nasogastric_tube',
                'nasogastric_reflux',
                'rectal_exam_feces', 
                'abdomen',
                'abdomo_appearance']
for column in null_columns:
    remove_null_using_mode(df, column)  # Use 'column' instead of 'null_columns'
    print(f'Done {column}')

Done temp_of_extremities
Done peripheral_pulse
Done mucous_membrane
Done capillary_refill_time
Done pain
Done peristalsis
Done abdominal_distention
Done nasogastric_tube
Done nasogastric_reflux
Done rectal_exam_feces
Done abdomen
Done abdomo_appearance


In [21]:
df.isna().sum()

id                       0
surgery                  0
age                      0
hospital_number          0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
nasogastric_reflux_ph    0
rectal_exam_feces        0
abdomen                  0
packed_cell_volume       0
total_protein            0
abdomo_appearance        0
abdomo_protein           0
surgical_lesion          0
lesion_1                 0
lesion_2                 0
lesion_3                 0
cp_data                  0
outcome                  0
dtype: int64

In [22]:
x = df.iloc[:,1:28]
y = df.iloc[:,28:]

# Label Encoding

In [23]:
# lable encoding
from sklearn.preprocessing import LabelEncoder

categorical_columns = x.select_dtypes(include=['object']).columns

label_encoder = LabelEncoder()

for column in categorical_columns:
    x[column] = label_encoder.fit_transform(x[column].astype(str))


In [24]:
# Fit and transform the 'outcome' column
y['outcome'] = label_encoder.fit_transform(y['outcome'])
y.head()

,outcome
0,0
1,1
2,2
3,2
4,2


# scale the data

In [25]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the entire DataFrame
x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

# Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV

# Define the hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l2']
}

# Create a Logistic Regression classifier
lr_classifier = LogisticRegression(random_state=12)

# Create a GridSearchCV object with the classifier, hyperparameter grid, and cross-validation settings
grid_search = GridSearchCV(lr_classifier, param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(x, y.values.ravel())

# Get the best estimator (classifier with the best hyperparameters)
best_lr_classifier = grid_search.best_estimator_

# Perform cross-validation to evaluate the Logistic Regression model
cv_scores = cross_val_score(best_lr_classifier, x, y.values.ravel(), cv=5)

# # Evaluate the Logistic Regression model on the test set
# y_pred_cv_lr = best_lr_classifier.predict(x_test)
# y_pred_original_cv_lr = label_encoder.inverse_transform(y_pred_cv_lr)

# output = pd.DataFrame({'id ': test.id , 'outcome': y_pred_original_cv_lr})
# output.to_csv('submission.csv', index=False)

In [28]:
cv_scores.mean()

0.6404858299595142

# Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier
# Define the hyperparameter grid for Random Forest
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=12)

# Create a GridSearchCV object with the classifier, hyperparameter grid, and cross-validation settings
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(x, y.values.ravel())

# Get the best estimator (classifier with the best hyperparameters)
best_rf_classifier = grid_search.best_estimator_

# Perform cross-validation to evaluate the Random Forest model
cv_scores = cross_val_score(best_rf_classifier, x, y.values.ravel(), cv=5)

# # Evaluate the Random Forest model on the test set
# y_pred_cv_rf = best_rf_classifier.predict(x_test)
# y_pred_original_cv_rf = label_encoder.inverse_transform(y_pred_cv_rf)

# output = pd.DataFrame({'id': test.id , 'outcome': y_pred_original_cv_rf})
# output.to_csv('submission.csv', index=False)

In [30]:
cv_scores.mean()

0.7060728744939271

# Decision Tree

In [31]:
from sklearn.tree import DecisionTreeClassifier
# Define the hyperparameter grid for Decision Tree
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=12)

# Create a GridSearchCV object with the classifier, hyperparameter grid, and cross-validation settings
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(x, y.values.ravel())

# Get the best estimator (classifier with the best hyperparameters)
best_dt_classifier = grid_search.best_estimator_

# Perform cross-validation to evaluate the Decision Tree model
cv_scores = cross_val_score(best_dt_classifier, x, y.values.ravel(), cv=5)

# # Evaluate the Decision Tree model on the test set
# y_pred_cv_dt = best_dt_classifier.predict(x_test)
# y_pred_original_cv_dt = label_encoder.inverse_transform(y_pred_cv_dt)

# output = pd.DataFrame({'id': test.id , 'outcome': y_pred_original_cv_dt})
# output.to_csv('submission_dt.csv', index=False)

In [32]:
cv_scores.mean()

0.6129554655870445